# Try Li Plating Sub Models

In [15]:
import pybamm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import dfols
import signal
from scipy.integrate import solve_ivp
from scipy.fft import fft, fftfreq, fftshift
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
from scipy import interpolate
from stopit import threading_timeoutable as timeoutable
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from batfuns import *
plt.rcParams = set_rc_params(plt.rcParams)

eSOH_DIR = "../data/esoh/"
oCV_DIR = "../data/ocv/"
fig_DIR = "../figures/figures_model/"
res_DIR = "../data/results_model/"
# %matplotlib widget

In [16]:
parameter_values = get_parameter_values()

In [17]:
parameter_values.search('stripping')

Exchange-current density for stripping [A.m-2]	<function stripping_exchange_current_density_Mohtat2020 at 0x0000018D87B044C0>


In [18]:
spm = pybamm.lithium_ion.SPM(
    {
        "SEI": "ec reaction limited",
        "loss of active material": "stress-driven",
    }
)
spm_li = pybamm.lithium_ion.SPM(
    {
        "SEI": "ec reaction limited",
        "loss of active material": "stress-driven",
        "lithium plating": "irreversible",
    }
)

spm_li_r = pybamm.lithium_ion.SPM(
    {
        "SEI": "ec reaction limited",
        "loss of active material": "stress-driven",
        "lithium plating": "reversible",
    }
)

spm_li_pr = pybamm.lithium_ion.SPM(
    {
        "SEI": "ec reaction limited",
        "loss of active material": "stress-driven",
        "lithium plating": "partially reversible",
    }
)

spme = pybamm.lithium_ion.SPMe(
    {
        "SEI": "ec reaction limited",
        "loss of active material": "stress-driven",
    }
)
spme_li = pybamm.lithium_ion.SPMe(
    {
        "SEI": "ec reaction limited",
        "loss of active material": "stress-driven",
        "lithium plating": "irreversible",
    }
)
# spm.print_parameter_info()
param=spm_li.param

In [19]:
cell = 4
cell_no,dfe,dfe_0,dfo_0,N,N_0 = load_data(cell,eSOH_DIR,oCV_DIR)
eps_n_data,eps_p_data,c_rate_c,c_rate_d,dis_set,Temp,SOC_0 = init_exp(cell_no,dfe,spm,parameter_values)
print(Temp)

25


In [20]:
parameter_values.update(
    {
      "Negative electrode active material volume fraction": eps_n_data,
      "Positive electrode active material volume fraction": eps_p_data,
      "Positive electrode LAM constant proportional term [s-1]": 1.27152e-07,
      "Negative electrode LAM constant proportional term [s-1]": 1.27272e-06,
      "Positive electrode LAM constant exponential term": 1.1992,
      "Negative electrode LAM constant exponential term": 1.1992,
      "SEI kinetic rate constant [m.s-1]":  4.60788219e-16, #1.08494281e-16 , 
      "EC diffusivity [m2.s-1]": 4.56607447e-19,#8.30909086e-19,
      "SEI growth activation energy [J.mol-1]": 1.87422275e+04,#1.58777981e+04,
    #   "Exchange-current density for plating [A.m-2]":0.008,
      # "Lithium plating transfer coefficient": 0.7,
      # "Dead lithium decay constant [s-1]": 1E-4,
    },
    check_already_exists=False,
)

In [21]:
# c_rate_d = '2C'
# c_rate_c = '2C'
# c_rate_c ='C/5'

In [22]:
pybamm.set_logging_level("WARNING")
pybamm.set_logging_level("NOTICE")
pybamm.set_logging_level("INFO")
pybamm.set_logging_level("VERBOSE")
pybamm.set_logging_level("SPAM")
# experiment = pybamm.Experiment(
#     [
#         (
#          "Charge at "+c_rate_c+" until 4.2V", 
#          "Hold at 4.2V until C/50"
#          )
#     ],
#     termination="50% capacity",
# #     cccv_handling="ode",
# )

experiment = pybamm.Experiment(
    [
        (
         "Charge at "+c_rate_c+" until 4.2V", 
         "Hold at 4.2V until C/50",
         "Rest for 5 min",
         "Discharge at "+c_rate_d+" until 3V",
         )
    ],
    termination="50% capacity",
#     cccv_handling="ode",
)

In [23]:
sim_long = pybamm.Simulation(spm_li, experiment=experiment, parameter_values=parameter_values, 
                            solver=pybamm.CasadiSolver("safe"))
sol_long = sim_long.solve(initial_soc=0)
solution1 = sol_long


2022-11-14 14:25:43.707 - [INFO] replace_symbols.process_model(43): Start replacing symbols in Single Particle Model
2022-11-14 14:25:43.708 - [VERBOSE] replace_symbols.process_model(58): Replacing symbols in Variable(-0x51e758cd18fb49fd, Discharge capacity [A.h], children=[], domains={}) (rhs)
2022-11-14 14:25:43.711 - [VERBOSE] replace_symbols.process_model(58): Replacing symbols in Variable(0x2e5839020d662db1, X-averaged negative electrode active material volume fraction, children=[], domains={'primary': ['current collector']}) (rhs)
2022-11-14 14:25:43.714 - [VERBOSE] replace_symbols.process_model(58): Replacing symbols in Variable(0x2c043bed0a94815, X-averaged positive electrode active material volume fraction, children=[], domains={'primary': ['current collector']}) (rhs)
2022-11-14 14:25:43.717 - [VERBOSE] replace_symbols.process_model(58): Replacing symbols in Variable(0x58182918eb52dbf6, X-averaged negative particle concentration, children=[], domains={'primary': ['negative pa

2022-11-14 14:25:43.757 - [VERBOSE] replace_symbols.process_boundary_conditions(137): Replacing symbols in PrimaryBroadcast(-0x3a474b2d629d7a71, broadcast, children=['x-average(broadcast(((2.0 + 2.0 * (1.0 / Reference temperature [K]) * (Ambient temperature [K] - Reference temperature [K])) / Positive electrode electrons in reaction) * arcsinh(-(Current function [A] / Typical current [A]) * sign(Typical current [A]) / (Positive electrode thickness [m] / (Negative electrode thickness [m] + Separator thickness [m] + Positive electrode thickness [m])) / (3.0 * X-averaged positive electrode active material volume fraction / x-average(Positive particle radius [m]) / (3.0 * yz-average(x-average(Positive electrode active material volume fraction)) / yz-average(x-average(Positive particle radius [m])))) / (2.0 * (maximum(broadcast(1.0), 1e-08)) * Positive electrode exchange-current density [A.m-2] / (absolute(Typical current [A] / (Number of electrodes connected in parallel to make a cell * El

2022-11-14 14:25:43.761 - [VERBOSE] replace_symbols.process_boundary_conditions(137): Replacing symbols in PrimaryBroadcast(-0x5686f7e5efcd8de4, broadcast, children=['-x-average(broadcast(((2.0 + 2.0 * (1.0 / Reference temperature [K]) * (Ambient temperature [K] - Reference temperature [K])) / Negative electrode electrons in reaction) * arcsinh((Current function [A] / Typical current [A]) * sign(Typical current [A]) / (Negative electrode thickness [m] / (Negative electrode thickness [m] + Separator thickness [m] + Positive electrode thickness [m])) / (3.0 * X-averaged negative electrode active material volume fraction / x-average(Negative particle radius [m]) / (3.0 * yz-average(x-average(Negative electrode active material volume fraction)) / yz-average(x-average(Negative particle radius [m])))) / (2.0 * (maximum(broadcast(1.0), 1e-08)) * Negative electrode exchange-current density [A.m-2] / (absolute(Typical current [A] / (Number of electrodes connected in parallel to make a cell * El

2022-11-14 14:25:43.807 - [VERBOSE] replace_symbols.process_model(87): Replacing symbols in 'Leading-order x-averaged separator porosity' (variables)
2022-11-14 14:25:43.808 - [VERBOSE] replace_symbols.process_model(87): Replacing symbols in 'Leading-order x-averaged positive electrode porosity' (variables)
2022-11-14 14:25:43.809 - [VERBOSE] replace_symbols.process_model(87): Replacing symbols in 'Leading-order negative electrode porosity' (variables)
2022-11-14 14:25:43.810 - [VERBOSE] replace_symbols.process_model(87): Replacing symbols in 'Leading-order x-averaged negative electrode porosity' (variables)
2022-11-14 14:25:43.811 - [VERBOSE] replace_symbols.process_model(87): Replacing symbols in 'Porosity change' (variables)
2022-11-14 14:25:43.812 - [VERBOSE] replace_symbols.process_model(87): Replacing symbols in 'Separator porosity change' (variables)
2022-11-14 14:25:43.813 - [VERBOSE] replace_symbols.process_model(87): Replacing symbols in 'Positive electrode porosity change' (

2022-11-14 14:25:43.858 - [VERBOSE] replace_symbols.process_model(87): Replacing symbols in 'Negative electrode volume-averaged velocity [m.s-1]' (variables)
2022-11-14 14:25:43.859 - [VERBOSE] replace_symbols.process_model(87): Replacing symbols in 'Positive electrode volume-averaged acceleration' (variables)
2022-11-14 14:25:43.860 - [VERBOSE] replace_symbols.process_model(87): Replacing symbols in 'Positive electrode volume-averaged acceleration [m.s-1]' (variables)
2022-11-14 14:25:43.861 - [VERBOSE] replace_symbols.process_model(87): Replacing symbols in 'X-averaged positive electrode volume-averaged acceleration' (variables)
2022-11-14 14:25:43.862 - [VERBOSE] replace_symbols.process_model(87): Replacing symbols in 'X-averaged positive electrode volume-averaged acceleration [m.s-1]' (variables)
2022-11-14 14:25:43.863 - [VERBOSE] replace_symbols.process_model(87): Replacing symbols in 'Negative electrode volume-averaged acceleration' (variables)
2022-11-14 14:25:43.863 - [VERBOSE

...

2022-11-14 14:25:55.925 - [DEBUG] casadi_solver._run_integrator(612): Running CasADi integrator
2022-11-14 14:25:55.926 - [SPAM] casadi_solver._run_integrator(622): Calculating t_eval_shifted
2022-11-14 14:25:55.927 - [SPAM] casadi_solver._run_integrator(625): Finished calculating t_eval_shifted
2022-11-14 14:25:55.927 - [SPAM] casadi_solver._run_integrator(639): Finished preliminary setup for integrator run
2022-11-14 14:25:55.928 - [DEBUG] casadi_solver._run_integrator(649): Calling casadi integrator
2022-11-14 14:25:55.930 - [DEBUG] casadi_solver._run_integrator(653): Finished casadi integrator
2022-11-14 14:25:55.932 - [DEBUG] casadi_solver._solve_for_event(271): Solving for events
2022-11-14 14:25:55.933 - [VERBOSE] base_solver.step(1227): Finish stepping Single Particle Model (the solver successfully reached the end of the integration interval)
2022-11-14 14:25:55.934 - [VERBOSE] base_solver.step(1228): Set-up time: 77.849 ms, Step time: 45.968 ms (of which integration time: 11.8

2022-11-14 14:25:56.137 - [SPAM] casadi_solver._run_integrator(625): Finished calculating t_eval_shifted
2022-11-14 14:25:56.138 - [SPAM] casadi_solver._run_integrator(639): Finished preliminary setup for integrator run
2022-11-14 14:25:56.139 - [DEBUG] casadi_solver._run_integrator(649): Calling casadi integrator
2022-11-14 14:25:56.141 - [DEBUG] casadi_solver._run_integrator(653): Finished casadi integrator
2022-11-14 14:25:56.143 - [DEBUG] casadi_solver._solve_for_event(271): Solving for events
2022-11-14 14:25:56.144 - [DEBUG] casadi_solver.create_integrator(482): Creating CasADi integrator
2022-11-14 14:25:56.145 - [DEBUG] casadi_solver._run_integrator(612): Running CasADi integrator
2022-11-14 14:25:56.146 - [SPAM] casadi_solver._run_integrator(622): Calculating t_eval_shifted
2022-11-14 14:25:56.147 - [SPAM] casadi_solver._run_integrator(625): Finished calculating t_eval_shifted
2022-11-14 14:25:56.148 - [SPAM] casadi_solver._run_integrator(639): Finished preliminary setup for i

2022-11-14 14:25:56.254 - [DEBUG] casadi_solver._run_integrator(653): Finished casadi integrator
2022-11-14 14:25:56.255 - [DEBUG] casadi_solver._solve_for_event(271): Solving for events
2022-11-14 14:25:56.257 - [DEBUG] casadi_solver.create_integrator(482): Creating CasADi integrator
2022-11-14 14:25:56.257 - [DEBUG] casadi_solver._run_integrator(612): Running CasADi integrator
2022-11-14 14:25:56.258 - [SPAM] casadi_solver._run_integrator(622): Calculating t_eval_shifted
2022-11-14 14:25:56.260 - [SPAM] casadi_solver._run_integrator(625): Finished calculating t_eval_shifted
2022-11-14 14:25:56.261 - [SPAM] casadi_solver._run_integrator(639): Finished preliminary setup for integrator run
2022-11-14 14:25:56.262 - [DEBUG] casadi_solver._run_integrator(649): Calling casadi integrator
2022-11-14 14:25:56.264 - [DEBUG] casadi_solver._run_integrator(653): Finished casadi integrator
2022-11-14 14:25:56.267 - [DEBUG] casadi_solver._solve_for_event(271): Solving for events
2022-11-14 14:25:56

2022-11-14 14:25:56.581 - [DEBUG] solution.update(474): Post-processing Loss of capacity to lithium plating [A.h]
2022-11-14 14:25:56.584 - [DEBUG] solution.update(474): Post-processing Loss of capacity to lithium plating [A.h]
2022-11-14 14:25:56.586 - [DEBUG] solution.update(474): Post-processing X-averaged SEI thickness [m]
2022-11-14 14:25:56.588 - [DEBUG] solution.update(474): Post-processing X-averaged SEI thickness [m]
2022-11-14 14:25:56.591 - [DEBUG] solution.update(474): Post-processing Loss of lithium to SEI on cracks [mol]
2022-11-14 14:25:56.593 - [DEBUG] solution.update(474): Post-processing Loss of lithium to SEI on cracks [mol]
2022-11-14 14:25:56.597 - [DEBUG] solution.update(474): Post-processing Loss of capacity to SEI on cracks [A.h]
2022-11-14 14:25:56.601 - [DEBUG] solution.update(474): Post-processing Loss of capacity to SEI on cracks [A.h]
2022-11-14 14:25:56.614 - [INFO] parameter_values.process_model(381): Start setting parameters for ElectrodeSOH model
2022-1

2022-11-14 14:25:56.734 - [VERBOSE] base_solver.report(1425): Calculating jacobian for algebraic using CasADi
2022-11-14 14:25:56.737 - [VERBOSE] base_solver.report(1425): Converting rhs_algebraic to CasADi
2022-11-14 14:25:56.747 - [VERBOSE] base_solver.report(1425): Calculating jacobian for rhs_algebraic using CasADi
2022-11-14 14:25:56.753 - [INFO] base_solver.set_up(235): Finish solver set-up
2022-11-14 14:25:56.754 - [VERBOSE] base_solver._get_discontinuity_start_end_indices(980): No discontinuity events found
2022-11-14 14:25:56.754 - [VERBOSE] base_solver.solve(860): Calling solver for 0.0 < t < 0.0
2022-11-14 14:25:56.756 - [DEBUG] algebraic_solver.root_fun(108): Evaluating algebraic equations at t=0.0, L2-norm is 1.4775255634092885
2022-11-14 14:25:56.757 - [DEBUG] algebraic_solver.root_fun(108): Evaluating algebraic equations at t=0.0, L2-norm is 1.4775255634092885
2022-11-14 14:25:56.758 - [DEBUG] algebraic_solver.root_fun(108): Evaluating algebraic equations at t=0.0, L2-no

In [ ]:
# parameter_values.update(
#     {
#     #   "Exchange-current density for plating [A.m-2]":0.001,
#     #   "Lithium plating transfer coefficient": 0.5,
#     #   "Dead lithium decay constant [s-1]": 1E-4,
#         # "Exchange-current density for stripping [A.m-2]": 0.00005,
#     },
#     check_already_exists=False,
# )
# c_rate_c ='1.5C'
# experiment = pybamm.Experiment(
#     [
#         (
#          "Charge at "+c_rate_c+" until 4.2V", 
#          "Hold at 4.2V until C/50"
#          )
#     ],
#     termination="50% capacity",
# #     cccv_handling="ode",
# )

sim_long = pybamm.Simulation(spm_li_r, experiment=experiment, parameter_values=parameter_values, 
                            solver=pybamm.CasadiSolver("safe"))
sol_long = sim_long.solve(initial_soc=0)
solution2 = sol_long

2022-11-14 14:25:56.863 - [INFO] replace_symbols.process_model(43): Start replacing symbols in Single Particle Model
2022-11-14 14:25:56.865 - [VERBOSE] replace_symbols.process_model(58): Replacing symbols in Variable(-0x51e758cd18fb49fd, Discharge capacity [A.h], children=[], domains={}) (rhs)
2022-11-14 14:25:56.867 - [VERBOSE] replace_symbols.process_model(58): Replacing symbols in Variable(0x2e5839020d662db1, X-averaged negative electrode active material volume fraction, children=[], domains={'primary': ['current collector']}) (rhs)
2022-11-14 14:25:56.870 - [VERBOSE] replace_symbols.process_model(58): Replacing symbols in Variable(0x2c043bed0a94815, X-averaged positive electrode active material volume fraction, children=[], domains={'primary': ['current collector']}) (rhs)
2022-11-14 14:25:56.873 - [VERBOSE] replace_symbols.process_model(58): Replacing symbols in Variable(0x58182918eb52dbf6, X-averaged negative particle concentration, children=[], domains={'primary': ['negative pa

2022-11-14 14:25:56.922 - [VERBOSE] replace_symbols.process_boundary_conditions(137): Replacing symbols in PrimaryBroadcast(-0x3a474b2d629d7a71, broadcast, children=['x-average(broadcast(((2.0 + 2.0 * (1.0 / Reference temperature [K]) * (Ambient temperature [K] - Reference temperature [K])) / Positive electrode electrons in reaction) * arcsinh(-(Current function [A] / Typical current [A]) * sign(Typical current [A]) / (Positive electrode thickness [m] / (Negative electrode thickness [m] + Separator thickness [m] + Positive electrode thickness [m])) / (3.0 * X-averaged positive electrode active material volume fraction / x-average(Positive particle radius [m]) / (3.0 * yz-average(x-average(Positive electrode active material volume fraction)) / yz-average(x-average(Positive particle radius [m])))) / (2.0 * (maximum(broadcast(1.0), 1e-08)) * Positive electrode exchange-current density [A.m-2] / (absolute(Typical current [A] / (Number of electrodes connected in parallel to make a cell * El

2022-11-14 14:25:56.925 - [VERBOSE] replace_symbols.process_boundary_conditions(137): Replacing symbols in PrimaryBroadcast(-0x5686f7e5efcd8de4, broadcast, children=['-x-average(broadcast(((2.0 + 2.0 * (1.0 / Reference temperature [K]) * (Ambient temperature [K] - Reference temperature [K])) / Negative electrode electrons in reaction) * arcsinh((Current function [A] / Typical current [A]) * sign(Typical current [A]) / (Negative electrode thickness [m] / (Negative electrode thickness [m] + Separator thickness [m] + Positive electrode thickness [m])) / (3.0 * X-averaged negative electrode active material volume fraction / x-average(Negative particle radius [m]) / (3.0 * yz-average(x-average(Negative electrode active material volume fraction)) / yz-average(x-average(Negative particle radius [m])))) / (2.0 * (maximum(broadcast(1.0), 1e-08)) * Negative electrode exchange-current density [A.m-2] / (absolute(Typical current [A] / (Number of electrodes connected in parallel to make a cell * El

2022-11-14 14:25:56.956 - [VERBOSE] replace_symbols.process_model(87): Replacing symbols in 'Leading-order x-averaged separator porosity' (variables)
2022-11-14 14:25:56.957 - [VERBOSE] replace_symbols.process_model(87): Replacing symbols in 'Leading-order x-averaged positive electrode porosity' (variables)
2022-11-14 14:25:56.957 - [VERBOSE] replace_symbols.process_model(87): Replacing symbols in 'Leading-order negative electrode porosity' (variables)
2022-11-14 14:25:56.958 - [VERBOSE] replace_symbols.process_model(87): Replacing symbols in 'Leading-order x-averaged negative electrode porosity' (variables)
2022-11-14 14:25:56.958 - [VERBOSE] replace_symbols.process_model(87): Replacing symbols in 'Porosity change' (variables)
2022-11-14 14:25:56.959 - [VERBOSE] replace_symbols.process_model(87): Replacing symbols in 'Separator porosity change' (variables)
2022-11-14 14:25:56.960 - [VERBOSE] replace_symbols.process_model(87): Replacing symbols in 'Positive electrode porosity change' (

2022-11-14 14:25:57.003 - [VERBOSE] replace_symbols.process_model(87): Replacing symbols in 'Negative electrode volume-averaged velocity [m.s-1]' (variables)
2022-11-14 14:25:57.003 - [VERBOSE] replace_symbols.process_model(87): Replacing symbols in 'Positive electrode volume-averaged acceleration' (variables)
2022-11-14 14:25:57.004 - [VERBOSE] replace_symbols.process_model(87): Replacing symbols in 'Positive electrode volume-averaged acceleration [m.s-1]' (variables)
2022-11-14 14:25:57.005 - [VERBOSE] replace_symbols.process_model(87): Replacing symbols in 'X-averaged positive electrode volume-averaged acceleration' (variables)
2022-11-14 14:25:57.006 - [VERBOSE] replace_symbols.process_model(87): Replacing symbols in 'X-averaged positive electrode volume-averaged acceleration [m.s-1]' (variables)
2022-11-14 14:25:57.007 - [VERBOSE] replace_symbols.process_model(87): Replacing symbols in 'Negative electrode volume-averaged acceleration' (variables)
2022-11-14 14:25:57.007 - [VERBOSE

...

2022-11-14 14:26:06.744 - [DEBUG] casadi_solver._run_integrator(612): Running CasADi integrator
2022-11-14 14:26:06.744 - [SPAM] casadi_solver._run_integrator(622): Calculating t_eval_shifted
2022-11-14 14:26:06.745 - [SPAM] casadi_solver._run_integrator(625): Finished calculating t_eval_shifted
2022-11-14 14:26:06.746 - [SPAM] casadi_solver._run_integrator(639): Finished preliminary setup for integrator run
2022-11-14 14:26:06.747 - [DEBUG] casadi_solver._run_integrator(649): Calling casadi integrator
2022-11-14 14:26:06.750 - [DEBUG] casadi_solver._run_integrator(653): Finished casadi integrator
2022-11-14 14:26:06.757 - [DEBUG] base_solver.get_termination_reason(1271): Start post-processing events
2022-11-14 14:26:06.758 - [DEBUG] base_solver.get_termination_reason(1316): Finish post-processing events
2022-11-14 14:26:06.759 - [VERBOSE] base_solver.step(1227): Finish stepping Single Particle Model (event: Charge voltage cut-off [V] [experiment])
2022-11-14 14:26:06.759 - [VERBOSE] b

2022-11-14 14:26:07.014 - [DEBUG] casadi_solver._solve_for_event(271): Solving for events
2022-11-14 14:26:07.016 - [DEBUG] casadi_solver.create_integrator(482): Creating CasADi integrator
2022-11-14 14:26:07.017 - [DEBUG] casadi_solver._run_integrator(612): Running CasADi integrator
2022-11-14 14:26:07.018 - [SPAM] casadi_solver._run_integrator(622): Calculating t_eval_shifted
2022-11-14 14:26:07.018 - [SPAM] casadi_solver._run_integrator(625): Finished calculating t_eval_shifted
2022-11-14 14:26:07.019 - [SPAM] casadi_solver._run_integrator(639): Finished preliminary setup for integrator run
2022-11-14 14:26:07.020 - [DEBUG] casadi_solver._run_integrator(649): Calling casadi integrator
2022-11-14 14:26:07.026 - [DEBUG] casadi_solver._run_integrator(653): Finished casadi integrator
2022-11-14 14:26:07.027 - [DEBUG] casadi_solver._solve_for_event(271): Solving for events
2022-11-14 14:26:07.029 - [DEBUG] casadi_solver.create_integrator(482): Creating CasADi integrator
2022-11-14 14:26:

2022-11-14 14:26:07.225 - [DEBUG] casadi_solver._run_integrator(612): Running CasADi integrator
2022-11-14 14:26:07.226 - [SPAM] casadi_solver._run_integrator(622): Calculating t_eval_shifted
2022-11-14 14:26:07.227 - [SPAM] casadi_solver._run_integrator(625): Finished calculating t_eval_shifted
2022-11-14 14:26:07.228 - [SPAM] casadi_solver._run_integrator(639): Finished preliminary setup for integrator run
2022-11-14 14:26:07.229 - [DEBUG] casadi_solver._run_integrator(649): Calling casadi integrator
2022-11-14 14:26:07.231 - [DEBUG] casadi_solver._run_integrator(653): Finished casadi integrator
2022-11-14 14:26:07.232 - [DEBUG] casadi_solver._solve_for_event(271): Solving for events
2022-11-14 14:26:07.233 - [VERBOSE] base_solver.step(1227): Finish stepping Single Particle Model (the solver successfully reached the end of the integration interval)
2022-11-14 14:26:07.235 - [VERBOSE] base_solver.step(1228): Set-up time: 86.271 ms, Step time: 50.402 ms (of which integration time: 19.7

2022-11-14 14:26:07.494 - [SPAM] casadi_solver._run_integrator(625): Finished calculating t_eval_shifted
2022-11-14 14:26:07.495 - [SPAM] casadi_solver._run_integrator(639): Finished preliminary setup for integrator run
2022-11-14 14:26:07.496 - [DEBUG] casadi_solver._run_integrator(649): Calling casadi integrator
2022-11-14 14:26:07.498 - [DEBUG] casadi_solver._run_integrator(653): Finished casadi integrator
2022-11-14 14:26:07.499 - [DEBUG] casadi_solver._solve_for_event(271): Solving for events
2022-11-14 14:26:07.501 - [DEBUG] casadi_solver.create_integrator(482): Creating CasADi integrator
2022-11-14 14:26:07.502 - [DEBUG] casadi_solver._run_integrator(612): Running CasADi integrator
2022-11-14 14:26:07.502 - [SPAM] casadi_solver._run_integrator(622): Calculating t_eval_shifted
2022-11-14 14:26:07.504 - [SPAM] casadi_solver._run_integrator(625): Finished calculating t_eval_shifted
2022-11-14 14:26:07.504 - [SPAM] casadi_solver._run_integrator(639): Finished preliminary setup for i

2022-11-14 14:26:07.591 - [DEBUG] casadi_solver._run_integrator(653): Finished casadi integrator
2022-11-14 14:26:07.592 - [DEBUG] casadi_solver._solve_for_event(271): Solving for events
2022-11-14 14:26:07.594 - [DEBUG] casadi_solver.create_integrator(482): Creating CasADi integrator
2022-11-14 14:26:07.595 - [DEBUG] casadi_solver._run_integrator(612): Running CasADi integrator
2022-11-14 14:26:07.596 - [SPAM] casadi_solver._run_integrator(622): Calculating t_eval_shifted
2022-11-14 14:26:07.596 - [SPAM] casadi_solver._run_integrator(625): Finished calculating t_eval_shifted
2022-11-14 14:26:07.597 - [SPAM] casadi_solver._run_integrator(639): Finished preliminary setup for integrator run
2022-11-14 14:26:07.598 - [DEBUG] casadi_solver._run_integrator(649): Calling casadi integrator
2022-11-14 14:26:07.600 - [DEBUG] casadi_solver._run_integrator(653): Finished casadi integrator
2022-11-14 14:26:07.602 - [DEBUG] casadi_solver._solve_for_event(271): Solving for events
2022-11-14 14:26:07

In [ ]:
# parameter_values.update(
#     {
#     #   "Exchange-current density for plating [A.m-2]":0.002,
#     #   "Lithium plating transfer coefficient": 0.7,
#       # "Dead lithium decay rate [s-1]": 1E-4,
#     },
#     check_already_exists=False,
# )

sim_long = pybamm.Simulation(spm_li_pr, experiment=experiment, parameter_values=parameter_values, 
                            solver=pybamm.CasadiSolver("safe"))
sol_long = sim_long.solve(initial_soc=0)
solution3 = sol_long

In [ ]:
t1 =  solution1["Time [s]"].entries
V1 =  solution1["Terminal voltage [V]"].entries
I1 =  solution1["Current [A]"].entries
Q1 = -solution1['Discharge capacity [A.h]'].entries
Q_Li1 = solution1["Loss of capacity to lithium plating [A.h]"].entries
I_Li1 = solution1["X-averaged lithium plating interfacial current density"].entries

In [ ]:
t2 =  solution2["Time [s]"].entries
V2 =  solution2["Terminal voltage [V]"].entries
I2 =  solution2["Current [A]"].entries
Q2 = -solution2['Discharge capacity [A.h]'].entries
Q_Li2 = solution2["Loss of capacity to lithium plating [A.h]"].entries
I_Li2 = solution2["X-averaged lithium plating interfacial current density"].entries

In [ ]:
t3 =  solution3["Time [s]"].entries
V3 =  solution3["Terminal voltage [V]"].entries
I3 =  solution3["Current [A]"].entries
Q3 = -solution3['Discharge capacity [A.h]'].entries
Q_Li3 = solution3["Loss of capacity to lithium plating [A.h]"].entries
I_Li3 = solution3["X-averaged lithium plating interfacial current density"].entries

In [ ]:
linestyles = ["dashed", "dotted", "solid"]
colors = ["tab:purple", "tab:cyan", "tab:red", "tab:green", "tab:blue"]

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(6, 8))
ax1 = axes.flat[0]
ax1.plot(t1,V1,'b')
ax1.plot(t2,V2,'r--')
ax1.plot(t3,V3,'g--')
ax1.set_title('Voltage [V]')
ax2 = axes.flat[1]
ax2.plot(t1,Q_Li1,'b')
ax2.plot(t2,Q_Li2,'r--')
ax2.plot(t3,Q_Li3,'g--')
ax2.set_title('Capacity lost due to plating [Ah]')
ax3=axes.flat[2]
ax3.plot(t1,I_Li1,'b')
ax3.plot(t2,I_Li2,'r--')
ax3.plot(t3,I_Li3,'g--')
ax3.set_title('Lithium plating interfacial current density')
# ax1.legend(['baseline',r'decreasing $\alpha_{pl}$',r'Increasing $k_{0,pl}$'])
ax1.legend(['Irreversible ','Reversible','Partially Reversible'])
ax1.legend(['C/5','1.5C'])
fig.tight_layout()
plt.savefig(fig_DIR+'plating_models_parameters'+'.png')

In [ ]:
dasasdasd

In [ ]:
parameter_values.update(
    {
      "Exchange-current density for plating [A.m-2]":0.008,
      "Lithium plating transfer coefficient": 0.7,
    #   "Dead lithium decay constant [s-1]": 1E-4,
        # "Exchange-current density for stripping [A.m-2]": 0.00005,
    },
    check_already_exists=False,
)
pybamm.set_logging_level("WARNING")
experiment = pybamm.Experiment(
    [
        ("Discharge at "+c_rate_d+dis_set,
         "Rest for 5 min",
         "Charge at "+c_rate_c+" until 4.2V", 
         "Hold at 4.2V until C/50")
    ] *dfe.N.iloc[-1],
    # ] *40,
    termination="50% capacity",
#     cccv_handling="ode",
)
all_sumvars_dict_spm = cycle_adaptive_simulation(spm, parameter_values, experiment,SOC_0, save_at_cycles=1)
all_sumvars_dict_spm_li = cycle_adaptive_simulation(spm_li, parameter_values, experiment,SOC_0, save_at_cycles=1)
def plotc2(all_sumvars_dict1,all_sumvars_dict2,esoh_data):
    esoh_vars = ["x_100", "y_0", "C_n", "C_p", "Capacity [A.h]", "Loss of lithium inventory [%]"]
    fig, axes = plt.subplots(3,2,figsize=(7,7))
    for k, name in enumerate(esoh_vars):
        ax = axes.flat[k]
        ax.plot(all_sumvars_dict1["Cycle number"],all_sumvars_dict1[name],"b.")
        ax.plot(all_sumvars_dict2["Cycle number"],all_sumvars_dict2[name],"rv")
        ax.plot(esoh_data["N"],esoh_data[name],"kx")
        ax.set_title(split_long_string(name))
        if k ==2 or k==3:
            ax.set_ylim([3,6.2])
        if k>3:
            ax.set_xlabel("Cycle number")
    fig.legend(["No Plating", "With Plating" , "Data"], 
           loc="lower center",bbox_to_anchor=[0.5,-0.05], ncol=1, fontsize=11)
    fig.tight_layout()
    return fig
plotc2(all_sumvars_dict_spm,all_sumvars_dict_spm_li,dfe)
plt.savefig(fig_DIR+"try_plating_cell_"+cell_no+"_2.png")

In [2]:
pip install stopit


  Using cached stopit-1.1.2.tar.gz (18 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11939 sha256=257b6aa71226e6d8f8db99ae7536444ff6cba6d5330e9e8bbbfddb527803c58f
  Stored in directory: c:\users\hamidreza\appdata\local\pip\cache\wheels\3f\2c\f9\411eea50089f1313eb80637f6107ec839721f6e36679050160
Successfully built stopit
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
